In [1]:
!pip install osmnx
!pip install matplotlib
!pip install geopandas
!pip install folium

In [2]:
import folium
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [3]:

def is_bike_friendly(edge):
    return (
        edge.get('highway') == 'cycleway' or
        'cycleway' in edge or
        edge.get('cyclelane') in ('lane', 'track', 'opposite', 'opposite_lane', 'opposite_track')
    )

# Define the place and create a graph from OSM data
place_name = "Münster, Germany"
G = ox.graph_from_place(place_name, network_type='all')

# Get bus stops and bike-friendly paths
tags = {'highway': 'bus_stop'}
bus_stops = ox.geometries_from_place(place_name, tags=tags)

# Get bicycle parking data
bicycle_parking_tags = {'amenity': 'bicycle_parking'}
bicycle_parkings = ox.geometries_from_place(place_name, tags=bicycle_parking_tags)

# Get the edges and nodes from the graph
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)

# Get bike-friendly edges and make a copy to avoid SettingWithCopyWarning
bike_friendly_edges = edges[edges.apply(is_bike_friendly, axis=1)].copy()

# Reproject the bike-friendly edges to UTM zone 32N for accurate length calculation
utm_crs = 'EPSG:32632'
bike_friendly_edges = bike_friendly_edges.to_crs(utm_crs)

# Calculate the length of each bike-friendly edge in meters
bike_friendly_edges['length_m'] = bike_friendly_edges.geometry.length

# Sum the lengths to get the total length in meters
total_length_m = bike_friendly_edges['length_m'].sum()

# Convert the total length from meters to kilometers
total_length_km = total_length_m / 1000
print(f"Total length of bike-friendly edges: {total_length_km} km")

nodes = ox.graph_to_gdfs(G, nodes=True, edges=False)

# Convert to the same CRS and create buffers
bus_stops = bus_stops.to_crs('EPSG:3395')
bus_stops_buffer = bus_stops.buffer(20)  # 20 meters buffer for bus stops
bike_friendly_edges = bike_friendly_edges.to_crs('EPSG:3395')
bike_friendly_buffer = bike_friendly_edges.buffer(20)  # 20 meters buffer for bike paths
nodes = nodes.to_crs('EPSG:3395')

# Find nodes near bus stops and bike paths
near_bus_stops = nodes[nodes.geometry.apply(lambda x: bus_stops_buffer.contains(x).any())]
near_bike_paths = nodes[nodes.geometry.apply(lambda x: bike_friendly_buffer.contains(x).any())]
near_both = near_bus_stops[near_bus_stops.index.isin(near_bike_paths.index)]

# Convert back to the original CRS for plotting
near_both = near_both.to_crs('EPSG:4326')
bike_friendly_edges = bike_friendly_edges.to_crs('EPSG:4326')
bus_stops = bus_stops.to_crs('EPSG:4326')

# Create a base map
map = folium.Map(location=[51.9625, 7.6256], zoom_start=13)

# Add bike-friendly paths to the map
for _, row in bike_friendly_edges.iterrows():
    line = folium.PolyLine(locations=[(lat, lon) for lon, lat in row['geometry'].coords], color='green', weight=5)
    map.add_child(line)

# Add intersections near both bus stops and bike paths
number_of_danger_points = len(near_both)
for _, row in near_both.iterrows():
    folium.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=7, 
        color='red', 
        fill=True,
        tooltip=f'Danger Point - Total: {number_of_danger_points}'
    ).add_to(map)

# Add bicycle parking to the map
for _, row in bicycle_parkings.iterrows():
    # Check if the geometry is a Point
    if isinstance(row.geometry, Point):
        location = [row.geometry.y, row.geometry.x]
    # If it's a Polygon, use the centroid
    elif isinstance(row.geometry, Polygon):
        centroid = row.geometry.centroid
        location = [centroid.y, centroid.x]
    else:
        continue  # Skip if the geometry type is neither Point nor Polygon

    folium.Marker(
        location=location,
        icon=folium.Icon(icon='bicycle', color='blue'),
        tooltip='Bicycle Parking'
    ).add_to(map)

print(f"Total number of danger points: {number_of_danger_points}")

number_of_bike_spots = len(bicycle_parkings)
print(f"Total number of bicycle parking: {number_of_bike_spots}")

# Display the map
map


/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_26826/1689060862.py:14: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bus_stops = ox.geometries_from_place(place_name, tags=tags)
/var/folders/vz/b1smgzt13_dbs5c1rbj4m3dh0000gn/T/ipykernel_26826/1689060862.py:18: UserWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in a future release.
  bicycle_parkings = ox.geometries_from_place(place_name, tags=bicycle_parking_tags)


Total length of bike-friendly edges: 85.99383023147055 km
Total number of danger points: 97
Total number of bicycle parking: 917
